### Moving Averaging

The advantage of Moving Averaging is that they are less prone to rampant loss shifts or irregular data representation in the latest batch. It gives a smooothened and a more genral idea of the model training until some point.

### Stocastic Averaging

Stocastic Weight Averaging converges to wider optimas. By doing so, it resembles geometric ensembeling. SWA is a simple method to improve model performance when used as a wrapper around other optimizers and averaging results from different points of trajectory of the inner optimizer.

In [1]:
import os

import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

### Build Model

In [2]:
def create_model(opt):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),                         
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=opt,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

    return model

In [3]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)

test_images, test_labels = test

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
# We'll be comparing Optimizers 
sgd = tf.keras.optimizers.SGD(0.01)
moving_avg_sgd = tfa.optimizers.MovingAverage(sgd)
stocastic_avg_sgd = tfa.optimizers.SWA(sgd)

In [5]:
# Both MovingAverage and StocasticAverage optimers use ModelAverageCheckpoint.

checkpoint_path = "./training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                 save_weights_only=True,
                                                 verbose=1)
avg_callback = tfa.callbacks.AverageModelCheckpoint(filepath=checkpoint_dir, 
                                                    update_weights=True)

### Train and Compare

In [6]:
# Build Model
model = create_model(sgd)

# Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[cp_callback])

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1874/1875 [============================>.] - ETA: 0s - loss: 0.7842 - accuracy: 0.7354
Epoch 00001: saving model to ./training
1875/1875 [==============================] - 6s 3ms/step - loss: 0.7842 - accuracy: 0.7354
Epoch 2/5
1864/1875 [============================>.] - ETA: 0s - loss: 0.5065 - accuracy: 0.8250
Epoch 00002: saving model to ./training
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5060 - accuracy: 0.8250
Epoch 3/5
1863/1875 [============================>.]

In [7]:
# Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

313/313 - 1s - loss: 76.1345 - accuracy: 0.8134
Loss : 76.134521484375
Accuracy : 0.8133999705314636


In [8]:
# Build Model
model = create_model(moving_avg_sgd)

# Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[avg_callback])

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1863/1875 [============================>.] - ETA: 0s - loss: 0.7611 - accuracy: 0.7480WARNING:tensorflow:From /home/tanmay/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property shou

In [9]:
# Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

313/313 - 1s - loss: 76.1345 - accuracy: 0.8134
Loss : 76.134521484375
Accuracy : 0.8133999705314636


In [10]:
# Build Model
model = create_model(stocastic_avg_sgd)

# Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[avg_callback])

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1875/1875 [==============================] - 9s 5ms/step - loss: 0.7905 - accuracy: 0.7414
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5609 - accuracy: 0.8084
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5309 - accuracy: 0.8176
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5144 - accuracy: 0.8225
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5060 - accuracy: 0.8251


In [11]:
# Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

313/313 - 1s - loss: 76.1345 - accuracy: 0.8134
Loss : 76.134521484375
Accuracy : 0.8133999705314636
